# Notebook for Lab 1 - Symmetric Encryption
# MADE BY : ALEJANDRO FDEZ-PANIAGUA AND ILAN HALIOUA

## General goal
In this lab you’ll get familiarized with programming (*using*) symmetric encryption algorithms using a modern library. The lab makes use of Python language but it could also be done with Java or C-like languages. 

## Specific goals
- Make use of different data representation formats (byte/ascii, string, hexadecimal, b64). Students should be able to recognize each of the formats and translate from one format to another.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages (strings) with block ciphers (AES, Camellia) operated under different modes (ECB, CBC and CTR) and considering padding issues.
- Make use of a robust pseudo random number generator function/algorithm to generate keys, initialization vectors and nonces.
- Understand how modifications in the ciphertext affect the decryption result when different operation modes are used.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages with stream ciphers (ChaCha20).
- Make use of the pylfsr library to generate a keystream from a LFSR.

## Modules
Cryptography library includes both high level recipes and low-level interfaces to common cryptographic algorithms such as symmetric ciphers, message digests, and key derivation functions. For more details you can consult the following link: https://pypi.org/project/cryptography. For the moment, we are going to directly use the primitives, forgetting about Fernet constructions. You’ll may find useful to have these links as reference:
- Links to the documentation of the specific sections at stake in this lab:
- Symmetric encryption: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/
- Symmetric padding: https://cryptography.io/en/latest/hazmat/primitives/padding/
- Link to info about os library: https://docs.python.org/3/library/os.html
- Link to info about base64 library: https://docs.python.org/3/library/base64.html
- Link info about pylfsr library: https://pypi.org/project/pylfsr/
- Link to your very first encryption/decryption code snippet example: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher

## Lab assignment and assessment
You are given one Python script/notebook, that contains the headers of all the functions you should develop at the end of this lab. Next, you’ll be guided through several tasks you’ll have to complete in order to do so.

Lab 1 assessment will be performed with a test/quiz/exam at the end of the course (along the other lab assessments).

# 0. Modules installation and imports

In [1]:
!pip install cryptography
!pip install pylfsr





  Using cached pylfsr-1.0.6-py3-none-any.whl (19 kB)


  Using cached numpy-1.23.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)


  Using cached matplotlib-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)


  Using cached fonttools-4.37.4-py3-none-any.whl (960 kB)


  Using cached contourpy-1.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (295 kB)


  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)



  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


  Using cached Pillow-9.2.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.2 MB)







In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

from cryptography.hazmat.primitives import padding

from pylfsr import LFSR

import base64

import os 

import secrets

# 1. Symmetric Block Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric block ciphers. First, you’ll use AES cipher operated under several confidentiality operation modes, and then you’ll be prompted to use other symmetric ciphers (CAMELLIA).
Read AND USE AS REFERENCE the information provided in the documentation web page of the pyca/cryptography library for symmetric encryption: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/ 

## 1.1 BYTE-LIKE DATA

### Task 1.1.1 From byte-like to hex and b64

First, let’s see a primer in dealing with binary data in Python, as it is a very frequent data format when using cryptography. The code lines for defining the key, the initialization vector and the text that you’ll use in your first encryption are given next:

In [3]:
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message'

Notice the preceding “b” in the previous data in single quotes. It is used to transform the provided (“text”-like) data into binary format (following ASCII codification https://en.wikipedia.org/wiki/ASCII). That is, the code will treat these data as binary. Each byte can be specified by:
- the ASCII character corresponding to that byte if it is printable, 
- the hexadecimal representation of the byte, preceding the 2-hex digits by “\x”.

Some examples: 
- b’MN’ corresponds to binary data 4d4e <sub>(16</sub> = 0100 1101 0100 1110 <sub>(2</sub> 
- b’\xa1\xf7’ corresponds to binary data a1f7 <sub>(16</sub> = 1010 0001 1111 0111 <sub>(2</sub>

You can get the hexadecimal representation of binary data using the built-in Python function <code>hex()</code> as follows (note: <code>hex()</code> belongs to the set of Python built-in functions (https://docs.python.org/3/library/functions.html):

In [4]:
binarydata = b'MN\\\xde\xa1' 
hex_binarydata = binarydata.hex() 
print("hex_binarydata =", hex_binarydata)

hex_binarydata = 4d4e5cdea1


You can get back your bytes from their hex representation as follows using <code>bytes.fromhex()</code> (https://docs.python.org/3/library/stdtypes.html?highlight=fromhex#bytes.fromhex):

In [5]:
retrieved_binarydata = bytes.fromhex(hex_binarydata) 
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


Other codification that can be used to deal in a print-friendly manner with binary data is B64 (https://en.wikipedia.org/wiki/Base64). 

An option is to use the Python module base64 (https://docs.python.org/3/library/base64.html). In order to use it you have to import the module in your code (you should have done it in section 0) and use <code>base64.b64encode()</code> as follows:

In [6]:
# import base64 

b64_binarydata = base64.b64encode(binarydata)
print("b64_binarydata =",b64_binarydata)

b64_binarydata = b'TU5c3qE='


As before, you can get your bytes back from B64 as follows using <code>base64.b64decode()</code>:

In [7]:
retrieved_binarydata = base64.b64decode(b64_binarydata)
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


#### TASK
<mark>Represent the given key, IV and message text in hexadecimal and B64 and check that you understand how binary data is provided and represented. Undo the hex and B64 representations by retrieving back the bytes and print the results.</mark>

In [5]:
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message'
key_hex = key.hex()
message_hex = message.hex()
iv_hex = iv.hex()
key_64 = base64.b64encode(key)
message_64 = base64.b64encode(message)
iv_64 = base64.b64encode(iv)

print("Data in hexadecimal base : ")
print("key",key_hex, "back: ", bytes.fromhex(key_hex))
print("message",message_hex, "back: ", bytes.fromhex(message_hex))
print("iv",iv_hex, "back: ", bytes.fromhex(iv_hex))
print("Data in base64 : ")
print("key",key_64, "back : ", base64.b64decode(key_64))
print("message",message_64, "back : ", base64.b64decode(message_64))
print("iv",iv_64,"back : ", base64.b64decode(iv_64))




Data in hexadecimal base : 

key 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a back:  b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

message 6120736563726574206d657373616765 back:  b'a secret message'

iv 7fb357841a394b4e7eaf73fc9fb6ccb8 back:  b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'

Data in base64 : 

key b'TU5c3qH3nM8QQageAA5lceg2HBIYphD7uV4KLDCia2o=' back :  b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

message b'YSBzZWNyZXQgbWVzc2FnZQ==' back :  b'a secret message'

iv b'f7NXhBo5S05+r3P8n7bMuA==' back :  b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'


## 1.2 AES256 - CBC MODE

### Task 1.2.1 Encrypt/decrypt with AES 256 CBC

Now, let’s encrypt a simple text (message) using the AES cipher with a 256-bit key and operating the cipher under CBC mode.

Read the information and take the code provided as example in the following web page section:

https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher 

Notice that Cipher is a general construction that can be instantiated with a specific algorithm, mode and backend (note: We will rely on the default backend but if you want to know more about backends, read this web page: https://cryptography.io/en/latest/hazmat/backends/). 

In this case we’ll use the classes <code>algorithms.AES(..)</code> and <code>modes.CBC(..)</code> for the first two parameters.

Notice also that in the example in the web page
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher
the key and iv data are randomly obtained.

#### TASK

<mark>Copy and change the code of the example (given at the web page of the library) so you use the key, iv and message given in Section 1.1. Leave the rest of the code unchanged. Additionally, insert calls to the <code>print()</code> function so the message, key, iv and the results of the encryption and decryption processes are printed. </mark>

In [6]:
# YOUR CODE HERE

import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message'
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(message) + encryptor.finalize()
decryptor = cipher.decryptor()
print("Plain text message : ", message)
print("CIPHERTEXT(message encrypted) >> ", ct)
print(ct.hex())
print(base64.b64encode(ct))
print("Message decrypted >> ")
decryptor.update(ct) + decryptor.finalize()


Plain text message :  b'a secret message'

CIPHERTEXT(message encrypted) >>  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'

3ba6e5853b5ec226ec077c03cc0dae4e

b'O6blhTtewibsB3wDzA2uTg=='

Message decrypted >> 


b'a secret message'

#### CHECK
Check your results with the following values:
<code>
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e
b64_ciphertext = b'O6blhTtewibsB3wDzA2uTg=='
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
b64_plaintext = b'YSBzZWNyZXQgbWVzc2FnZQ=='
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.2? </mark>

<mark>And the output ciphertext? </mark>

Notice that you may use the built-in function <code>len(..)</code> to get the length of an object (https://docs.python.org/3/library/functions.html#len).

In [7]:
# YOUR CODE HERE
print("The input data to encrypt has length : ", len(message))
print("The output data (encrypted message, ciphertext), has length: ", len(ct))

The input data to encrypt has length :  16

The output data (encrypted message, ciphertext), has length:  16


### Task 1.2.2 Adding padding to AES 256 CBC

Now let’s play with the length of the message to encrypt! In the previous exercise you encrypted a message that has an exact length of 16 characters, that is, the AES block size. Try to encrypt a longer message whose length in characters is not multiple of 16. You’ll get an error for sure! It is because if you need to encrypt a message with length not multiple of the cipher block size and you are using CBC operation mode, you need to add padding to your input so its length satisfies that condition.

Read the information provided in the documentation web page of the pyca/cryptography library on how the library deals with symmetric padding:
https://cryptography.io/en/latest/hazmat/primitives/padding/

Study the code in the example provided to illustrate the PKCS7 padding here:
https://cryptography.io/en/latest/hazmat/primitives/padding/#cryptography.hazmat.primitives.padding.PKCS7

Notice that you have to import the padding module from the pyca/cryptography library (if you have not done it in Section 0). Notice also that the correct order to combine padding-encryption and decryption-unpadding is the following one:
1. Pad the data to encrypt to obtain the padded data
2. Encrypt the padded data to obtain the encrypted text
3. --- (simulation of data exchange) ----
4. Decrypt the encrypted text to obtain the padded plaintext
5. Unpad the padded plaintext to obtain the original plaintext

#### TASK 
<mark>After studying it, incorporate PKCS7 padding in your previous code. </mark>

In [45]:
# YOUR CODE HERE
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message here'#plain text message length not multiple of 16 bytes, key length
padder = padding.PKCS7(128).padder()
padded_data = padder.update(message)
padded_data += padder.finalize()
print("Padded data : ",padded_data)
print("length : ", len(padded_data))
#now, encrypt the padded message using Aes as done before...
cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(padded_data) + encryptor.finalize()
decryptor = cipher.decryptor()
print("Encrypted padded data : ",ct)
padded_message = decryptor.update(ct) + decryptor.finalize()
unpadder = padding.PKCS7(128).unpadder()
plaintext = unpadder.update(padded_data)
plaintext = plaintext + unpadder.finalize()
print("Decrypted padded data : ",padded_message)
print("Decrypted unpadded data : ", plaintext)





Padded data :  b'a secret message here\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'

length :  32

Encrypted padded data :  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN\x10A\x05#\xdd $\x07q#\x05\x9d\x9aS:\xb6'

Decrypted padded data :  b'a secret message here\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'

Decrypted unpadded data :  b'a secret message here'


#### CHECK
Your previous AES256-CBC code, updated to include PKCS7 padding, with the data provided in Section 1.1 (key, iv and message), should provide the result shown next:
<code>
message = b'a secret message'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.2? </mark>

<mark>Which length in bytes has the output ciphertext in Task 1.2.2? </mark>

<mark>How many bytes have been added as PKCS7 padding? </mark>

<mark>Which value has been used in the bytes added as padding? </mark>

In [ ]:
# YOUR CODE HERE
#the input data tp encrypt had length 16 bytes = 128 bits (multiple of block length in AES , which is 16 bytes)
#the length in bytes of the ct is 32 since we called the padd function and it added 16 more bytes even though it was not neccessary since we coul encrypt it with 16 already
#16 bytes were added when padding the plain text message to later encrypt using AES with block size 16
#the binary value 10101010101010101010101010101010(32 more bits = 16 more bytes) has been added as padding to the message


YOUR ANSWER HERE

### Task 1.2.3 Checking AES 256 CBC PKCS7

#### TASK

<mark>Check that your code works correctly by encrypting <code>other_message</code>, a message with 25 characters (length not multiple of 16) and provided next, and the same <code>key</code> and <code>iv</code> already used. </mark>

In [46]:
# YOUR CODE HERE
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message not multiple of 16'#plain text message length not multiple of 16 bytes, key length
print(len(message))
padder = padding.PKCS7(128).padder()
padded_data = padder.update(message)
padded_data += padder.finalize()
print("Padded data : ",padded_data)
print("length : ", len(padded_data))
#now, encrypt the padded message using Aes as done before...
cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(padded_data) + encryptor.finalize()
decryptor = cipher.decryptor()
print("Encrypted padded data : ",ct)
padded_message = decryptor.update(ct) + decryptor.finalize()
unpadder = padding.PKCS7(128).unpadder()
plaintext = unpadder.update(padded_data)
plaintext = plaintext + unpadder.finalize()
print("Decrypted padded data : ",padded_message)
print("Decrypted unpadded data : ", plaintext)






35

Padded data :  b'a secret message not multiple of 16\r\r\r\r\r\r\r\r\r\r\r\r\r'

length :  48

Encrypted padded data :  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN\\\xe2\xa1\xe5\x1d\x94p2Xa\xd4\xa1\xb7\x17\xcc\xf1G\xfaq\xb8?\x19\x06\xcc\x95\xd1\x0e\xa5\xdb\t?e'

Decrypted padded data :  b'a secret message not multiple of 16\r\r\r\r\r\r\r\r\r\r\r\r\r'

Decrypted unpadded data :  b'a secret message not multiple of 16'


#### CHECK
You should get the following results:
<code>
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.3?</mark>

<mark>Which length in bytes has the output ciphertext in Task 1.2.3?</mark>

<mark>How many bytes have been added as PKCS7 padding?</mark> 

<mark>Which value has been used in the bytes added as padding?</mark>

In [ ]:
# YOUR CODE HERE
#the input data before has length 25 Bytes which is not a valid length to encrypt using 16 B block size AES so padded is needed to get to 32 Bytes at least
#the output ct has length 25 Bytes since the ct has been decrypted and unpadded leading to the original message == plaintext
#7 Bytes have been added as padding to the 25 Bytes Plain text, in order to be able to separate the data into blocks of size 16 and encrypt using AES
#the hexadecmal value 0X07070707070707 which is 0111000001110000011100000111000001110000011100000111 in binary

YOUR ANSWER HERE

### Task 1.2.4 Wrapping AES 256 CBC PKCS7 encryption/decryption in functions

#### TASK

<mark>Let’s wrap your AES256-CBC-PKCS7 code in two functions, one for encryption and another one for decryption. Next you have the headers that you should use for these two functions.</mark>

In [76]:
def aes256_cbc_pkcs7_encrypt(data_to_encrypt, key, iv):
    #We will basically do the following :
    #Step 1: Pad the data to have a length multiple of 16 Bytes
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data_to_encrypt)
    padded_data += padder.finalize()
    #Step 2: Encrypt the padded data to encrypt using AES with 16 Bytes blocks size
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext

def aes256_cbc_pkcs7_decrypt(encrypted_data, key, iv):
    #We will basically do the following :
    #Step 1: Decrypt the padded encrypted_data 
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    #Step 2 : Unpad the padded part
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext)
    plaintext = message +unpadder.finalize()
    return message


key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'My name is Alejandro'#plain text message length not multiple of 16 bytes, key length
print(len(message))
ct = aes256_cbc_pkcs7_encrypt(message, key, iv)
print("Ciphertext : ", ct)
plaintext = aes256_cbc_pkcs7_decrypt(ct, key, iv)
print("Plaintext decrypted from ciphertext : ", plaintext)


20

Ciphertext :  b'\xd2\xfe%\xf6\xa1Ti\xd5\x81\x14KC*\x7f\x81\x03\xa1\\1\xb4zw\xa0\xee\xa2\x11\xb3n\xf9j\xfe\x93'

Plaintext decrypted from ciphertext :  b'My name is Alejandro'


#### CHECK

Test your functions with the data and results of Task 1.2.3

### Task 1.2.5 Checking AES 256 CBC PKCS7 against CyberChef

#### TASK

<mark> Test again your AES256-CBC-PKCS7 function but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ . You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and CBC mode.  </mark>

## 1.3 AES256 - ECB MODE

Let’s now use the ECB operation mode. It is the simplest of all the confidentiality operation modes and does not need any parameter. You can consult the library documentation for this mode here: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.ECB

### Task 1.3.1 Encrypt/decrypt with AES 256 ECB PKCS7

#### TASK

<mark>Write a new encryption and decryption code based on your AES256-CBC-PKCS7 code but that makes use of the ECB mode instead of the CBC one. Wrap the code in two functions (headers given below).</mark>

In [81]:
def aes256_ecb_pkcs7_encrypt(data_to_encrypt, key):
    #We will basically do the following :
    #Step 1: Pad the data to have a length multiple of 16 Bytes
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data_to_encrypt)
    padded_data += padder.finalize()
    #Step 2: Encrypt the padded data to encrypt using AES with 16 Bytes blocks size
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext    

def aes256_ecb_pkcs7_decrypt(encrypted_data, key):
    #We will basically do the following :
    #Step 1: Decrypt the padded encrypted_data 
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    # Step 2: remove the padded part
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext)
    plaintext = plaintext +unpadder.finalize()
    return plaintext
    #### CHECK
message = b'My name is Alejandro Alejandro'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
ciphertext = aes256_cbc_pkcs7_encrypt(message, key)
print("Message : ", message)
print("length" , len(message))
print("Encrypted message (padded included) : ",ciphertext)
decrypted = aes256_ecb_pkcs7_decrypt(ciphertext, key)
print("Decrypted message (padded removed) : ",decrypted)


Message :  b'My name is Alejandro Alejandro'

length 30

Encrypted message (padded included) :  b';yl\n\xcdF;v\x90\xbf\x04\x91\xd0G\x84\xe0\x1c\xb8\x16\x9fG\xa6\x8f\xea\x1e\xdaw\x94p j\xad'

Decrypted message (padded removed) :  b'My name is Alejandro Alejandro'


You should obtain the following results if you encrypt <code>message</code> with the same <code>key</code> previously used.
<code>
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' 
length =  32
ciphertext = b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
hex_ciphertext = 69b0611b958f16d6ed31c6fca5abd5467c98fa0b8eabd4d78e8ff50a2126fdbb
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

### Task 1.3.2 Checking AES 256 ECB PKCS7 

#### TASK

<mark>Test your code now with the message <code>other_message</code>, with a length in char not multiple of 16. </mark>

In [82]:
other_message = b"This message is important" #25 chars that is 25 Bytes long, not multiple of 16 (next is 32 so it will pad another 7 Bytes to it)
ciphertext = aes256_ecb_pkcs7_encrypt(other_message, key)
print("Message : ", message)
print("Length : ", len(message))
print("Ciphertext : ", ciphertext, "\nwhere the length is : ", len(ciphertext))
decrypted = aes256_ecb_pkcs7_decrypt(ciphertext, key)
print("Decrypted message (padding removed): ", decrypted)
 

Message :  b'My name is Alejandro Alejandro'

Length :  30

Ciphertext :  b'\xb8{\xe1\x8e\x9fc\xfb\xaa\x13y\xb6\xf5VQW\xc4\xa1\xd0\xc1! .\xff\x9a\xb4\xc9X\x01\xc2\xc9\xf9\xd0' 

where the length is :  32

Decrypted message (padding removed):  b'This message is important'


#### CHECK

You should obtain the following results:
<code>
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07' 
length =  32
ciphertext = b'\xb8{\xe1\x8e\x9fc\xfb\xaa\x13y\xb6\xf5VQW\xc4\xa1\xd0\xc1! .\xff\x9a\xb4\xc9X\x01\xc2\xc9\xf9\xd0'
hex_ciphertext = b87be18e9f63fbaa1379b6f5565157c4a1d0c121202eff9ab4c95801c2c9f9d0
padded_plaintext = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
hex_padded_plaintext = 54686973206d65737361676520697320696d706f7274616e7407070707070707
plaintext = b'This message is important'
hex_plaintext = 54686973206d65737361676520697320696d706f7274616e74
</code>

### Task 1.3.3 Checking AES 256 ECB PKCS7 against CyberChef

#### TASK

<mark>Test again your AES256-ECB-PKCS7 function but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ . You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and ECB mode. </mark>

### Task 1.3.4 Generating cryptographically secure random bytes

Till now you have used fixed and given key and iv in your AES256 code. However, ideally, you should randomly generate these parameters each time you need them, at least in the first encryption step. Notice that it is not the same case in the decryption step, as here you must use the same parameters used in the encryption step if you want to correctly recover the plaintext.

Let’s learn first how to generate <ins>cryptographically secure</ins> random bytes.

Read the information provided in the web page section: https://cryptography.io/en/latest/random-numbers/#random-number-generation

Two possibilities are highlighted:

- Using the function <code>urandom(..)</code> from the os module (https://docs.python.org/3/library/os.html#os.urandom)

- Using the function <code>token_bytes(..)</code> from the secrets module (https://docs.python.org/3/library/secrets.html#secrets.token_bytes)

You should have imported both in Section 0.

#### TASK

<mark>Try both ways with the code shown next. Notice that each time you execute the code, new bytes are obtained.</mark>

In [63]:
# import os 
# import secrets

print(os.urandom(16)) # generating 16 random bytes
print(secrets.token_bytes(32))   # generating 32 random bytes

b'\x9aK\x15\xb7\x80R\xf39_\x8f\xb5!Q\\\xcf_'

b'\x89Z\x12F\x8b\xca\x802\x98\xbe\xbci\x96\xed\xe6\xa1\x1f\x88\xa2\xdb\x01\x0f^$\xa1XM\xee*\xa1S\x9d'


### Task 1.3.5 No-key AES 256 ECB PKCS7 and No-key_no-iv AES 256 CBC PKCS7 encryption functions

Let’s change the your AES256-ECB-PKCS7 encryption code so if no key is provided, the key is randomly generated. Both the ciphertext and the key should be provided as output. Use the code provided below 14 as guide to write a new function <code>aes256_ecb_pkcs7_encrypt_rnd(..)</code> that incorporates this functionality. 

Note that the function expects two parameters but by writing <code>key=None</code> in the function header for the second one, we allow its absence (that is, it is optional to call the function with a second parameter) and at the same time we provide a default value if it is not provided (in this case this default value is the <code>None</code> value). This means also that, if the key value is not provided, we have to provide a key value inside the function code (in the provided code <code>os.urandom(..)</code> is used to generate this value randomly). As the key value is needed for decryption, notice that we also return it together with the resulting ciphertext. 

The same happens with the initialization vector <code>iv</code> in the case of the CBC mode. Notice that in the case of AES cipher, the <code>iv</code> size is the same as the block size, that is, 16 bytes.

#### TASK

<mark>Re-write the encryption functions as explained taking the following headers and code as starting point.</mark>

In [90]:
def aes256_ecb_pkcs7_encrypt_rnd(data_to_encrypt, key=None): #recall that ECB op mode doesnt need the initialization vector ,iv
    if key is None:
        key = os.urandom(16) 
        #We will basically do the following :
        #Step 1: Pad the data to have a length multiple of 16 Bytes
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(data_to_encrypt)
        padded_data += padder.finalize()
        #Step 2: Encrypt the padded data to encrypt using AES with 16 Bytes blocks size
        cipher = Cipher(algorithms.AES(key), modes.ECB())
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext, key

def aes256_cbc_pkcs7_encrypt_rnd(data_to_encrypt, key=None, iv=None):
    #we set random 16 bytes value for the key and the iv here: 
    key = os.urandom(16)
    iv = os.urandom(16) #must be the same size
    #We will basically do the following :
    #Step 1: Pad the data to have a length multiple of 16 Bytes
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data_to_encrypt)
    padded_data += padder.finalize()
    #Step 2: Encrypt the padded data to encrypt using AES with 16 Bytes blocks size
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext, key, iv

#### CHECK

Once you have the code running, try to encrypt and decrypt the same message and check that different keys and ciphertext are obtained.

In [86]:
# CHECK ECB FUNCTION

print("# ECB FIRST RUN ")
ciphertext1, randomkey1 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex())
plaintext1 = aes256_ecb_pkcs7_decrypt(ciphertext1, randomkey1)
print("plaintext1 = ", plaintext1)

print("\n# ECB SECOND RUN")
ciphertext2, randomkey2 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex())
plaintext2 = aes256_ecb_pkcs7_decrypt(ciphertext2, randomkey2)
print("plaintext2 = ", plaintext2)

# ECB FIRST RUN 

hex_randomkey1 = c1834cc8cb0920a1fcecb10b31a70f25 

hex_ciphertext1 = 23491cb081079fa813153ebbf0d273f33ca1ec2e51537eb257f60f8e72ce852b

plaintext1 =  b'This message is important'



# ECB SECOND RUN

hex_randomkey2 = ffd624015316cb4783bc748db73dc706 

hex_ciphertext2 = f0abb377ac11a52dc922376d6e856ef2bcf505bf79ed9a225ccb6579d2fa38e8

plaintext2 =  b'This message is important'


In [91]:
# CHECK CBC FUNCTION

print("# CBC FIRST RUN")
ciphertext1, randomkey1, randomiv1= aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex(), "\nhex_randomiv1 =", randomiv1.hex())
plaintext1 = aes256_cbc_pkcs7_decrypt(ciphertext1, randomkey1, randomiv1)
print("plaintext1 = ", plaintext1)

print("\n# CBC SECOND RUN")
ciphertext2, randomkey2, randomiv2 = aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex(), "\nhex_randomiv2 =", randomiv2.hex())
plaintext2 = aes256_cbc_pkcs7_decrypt(ciphertext2, randomkey2, randomiv2)
print("plaintext2 = ", plaintext2)

# CBC FIRST RUN

hex_randomkey1 = a31af04e94cc223d9b88b8e5180a6ae6 

hex_ciphertext1 = 388e2a8a6d8f554ab9e7ac1e9bb91a4cac1d8ebb4a807db0624e7788c2e5705d 

hex_randomiv1 = cc251967e9aa6b550711f35ddf24dcd2

plaintext1 =  b'My name is Alejandro Alejandro'



# CBC SECOND RUN

hex_randomkey2 = 9908394368d83cb8c416db7ab31e7072 

hex_ciphertext2 = b5d6f4c8905b4f2aa61229d138383e8045725168e5180c22b25367d4474585b0 

hex_randomiv2 = 00f66bd25687e984d68b7bcbdb9d3e4d

plaintext2 =  b'My name is Alejandro Alejandro'


## 1.4 AES256 - CTR MODE

Let’s now use the CTR operation mode. You can consult the library documentation for this mode here: 
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.CTR 
Notice that in this case you do not need an initialization vector, but you need a nonce to initialize the counter. 

### Task 1.4.1 Encrypt/decrypt with AES 256 CTR

#### QUESTION

<mark>Review your notes on CTR operation mode. Does CTR operation mode need padding? That is, is a requirement that the length of the data to encrypt in bytes is multiple of the cipher block size?</mark>

## No, it doesnt require padding so the length of the data to encrypt is arbitrary

#### TASK

<mark>Write two new encryption and one decryption functions based on your AES256-CBC code but that makes use of the CTR mode instead of the CBC one. Notice that PKCS7 padding will NOT be applied.</mark>

In [96]:
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

def aes256_ctr_encrypt(data_to_encrypt, key, nonce):
    #We will basically do the following :
    #we encrypt directly without padding since its not a requirement
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data_to_encrypt) + encryptor.finalize()        
    return ciphertext


def aes256_ctr_decrypt(encrypted_data, key, nonce):
    #We will basically do the following :
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    return plaintext


def aes256_ctr_encrypt_rnd(data_to_encrypt, key=None, nonce=None):
    if key is None and nonce is None:
        #We will basically do the following :
        #get random key and iv
        key = us.random(16)
       #we encrypt directly without padding since its not a requirement
        cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(data_to_encrypt) + encryptor.finalize()        
    return ciphertext, key, nonce

#### CHECK

In [97]:
# CHECK YOUR CODE

nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

ciphertext1 = aes256_ctr_encrypt(other_message, key, nonce)
ciphertext2 = aes256_ctr_encrypt(other_message+other_message, key, nonce)

print("ciphertext1 =", ciphertext1.hex(), "\nlength ciphertext1 =", len(ciphertext1))
print("ciphertext2 =", ciphertext2.hex(), "\nlength ciphertext2 =", len(ciphertext2))

plaintext1 = aes256_ctr_decrypt(ciphertext1, key, nonce)
plaintext2 = aes256_ctr_decrypt(ciphertext2, key, nonce)

print("plaintext1 =", plaintext1)
print("plaintext2 =", plaintext2)

other_message : b'This message is important'

key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a

nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

hex_nonce = f42489ab7ba42d603ade71b56d1bd398

ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 

length ciphertext1 = 25

ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 

length ciphertext2 = 50

plaintext1 = b'This message is important'

plaintext2 = b'This message is importantThis message is important'


You should obtain the following results if you encrypt <code>other_message</code> and two times <code>other_message</code> with the same <code>key</code> previously used and the provided <code>nonce</code>.
<code>
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 
length ciphertext1 = 25
ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 
length  ciphertext2 = 50
plaintext1 = b'This message is important'
plaintext2 = b'This message is importantThis message is important'
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.4.1? And the output ciphertext? Is the length of the output ciphertext multiple of the cipher block size or not and why?</mark> 

The input data to encrypt ("This message is important") is 25 Bytes long which is the same as the output ciphertext one. This length is not a multiple of the block size since operation mode CTR doesnt depend on it.

### Task 1.4.2 Checking AES 256 CTR against CyberChef

#### TASK

<mark>Test again your AES256-CTR code but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ </mark>

## 1.5 ADDITIONAL ASPECTS OF OPERATION MODES

### Task 1.5.1 No-padding AES 256 ECB and CBC functions

You have already been questioned about the difference in the output lengths when different operation modes are used and when padding is added or not. Now you’ll study what happens when you intentionally modify one bit (or octet) of a ciphertext and then decrypt the modified ciphertext. 

In order to play with modifications in the ciphertext, it is better to avoid PKCS7 padding, as if modifications affect the padding, it will probably raise an error and you’ll not be able to study the effects of the modifications in the plaintext after decryption. 

#### TASK 

<mark>Write a new set of functions for AES256-CBC and AES256-ECB encryption and decryption that do not apply padding.</mark>

In [113]:
def aes256_cbc_encrypt(data_to_encrypt, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data_to_encrypt) + encryptor.finalize()    
    return ciphertext

def aes256_cbc_decrypt(encrypted_data, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()    
    return plaintext

def aes256_ecb_encrypt(data_to_encrypt, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data_to_encrypt) + encryptor.finalize()        
    return ciphertext

def aes256_ecb_decrypt(encrypted_data, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()    
    return plaintext


#### CHECK

Test your functions by encrypting messages with length in chars multiple of 16 and then decrypting the resulting ciphertext.

In [117]:
# CHECKING YOUR CODE
message = b'a secret message' #message multiple of 16 bytes long
print("length", len(message))
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
print("message to encrypt and decrypt ->", message)
print("key :", key)
print("hex_key :", key.hex())
print("iv :", iv)
print("hex_iv :", iv.hex())

print("AES256-CBC ciphertext: ", aes256_cbc_encrypt(message, key, iv))
print("AES256-CBC plaintext: ", aes256_cbc_decrypt(aes256_cbc_encrypt(message, key, iv), key, iv))

print("AES256-ECB ciphertext: ", aes256_ecb_encrypt(message, key))
print("AES256-ECB plaintext: ", aes256_ecb_decrypt(aes256_ecb_encrypt(message, key), key))

length 16

message to encrypt and decrypt -> b'a secret message'

key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a

iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'

hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8

AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'

AES256-CBC plaintext:  b'a secret message'

AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'

AES256-ECB plaintext:  b'a secret message'


With the previously used values for <code>message</code>, <code>key</code> and <code>iv</code>, you should obtain the following results:
<code>
message : b'a secret message'
key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8
AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
AES256-CBC plaintext:  b'a secret message'
AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'
AES256-ECB plaintext:  b'a secret message'
</code>

### Task 1.5.2 Effects of modifying ciphertext under different operation modes 

#### TASK

<mark>Now, analyse how the decrypted plaintext changes after modifying one bit of the ciphertext at different positions (one in the first block, another in the second block…).</mark>

Suggested steps:
- Select a message with at least 48-char length (remember that the length in char has to be multiple of 16)
- Select a key, and iv and nonce if necessary
- Encrypt the message operating the cipher with the selected mode
- Save the ciphertext in one variable (ciphertext)
- Modify the previous ciphertext in one bit and save the result in another variable (modified_ciphertext)
- Decrypt both the ciphertext and its modified version, saving the decryptions, respectively, in two variables plaintext and modified_plaintext
- Analyse how the ciphertext modification has affected the plaintext

In [124]:
# YOUR CODE HERE
#ANALYSIS OF INFLUENCE OF A BYTE VARIATION ON THE CIPHERTEXT IN THE DECRYPTION USING CBC
message = b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' #message multiple of 16, concretely 48 Bytes long(minimum here)
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
print("message is -> ",message,"\nlength", len(message),"Bytes")
ct = aes256_cbc_encrypt(message, key, iv) #consider no padding here since we will always give multiple of 16, >=48
modified_ct = ct


message is ->  b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' 

length 48 Bytes


#### QUESTION

<mark>How many bits (or bytes) are affected in each case (ECB, CBC, CTR)? Which are their positions? Can you explain the reasons in each case considering the course notes on operation modes?</mark>

YOUR ANSWER HERE

## 1.6 OTHER CIPHERS AND KEY SIZES

### Task 1.6.1 Camellia symmetric block cipher

#### TASK

<mark>Now, create new functions similar to the ones already coded but that in this case use the Camellia cipher and other key sizes. Consult the library documentation to know more about the Camellia cipher and available key sizes for both AES and Camellia.</mark>

In [137]:
def Camellia_cbc_encrypt(data_to_encrypt, key, iv):
    #We will basically do the following :
    #Step 1: Pad the data to have a length multiple of 16 Bytes
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data_to_encrypt)
    padded_data += padder.finalize()
    #Step 2: Encrypt the padded data to encrypt using AES with 16 Bytes blocks size
    cipher = Cipher(algorithms.Camellia(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext
def Camellia_cbc_decrypt(encrypted_data, key, iv):
    #We will basically do the following :
    #Step 1: Decrypt the padded encrypted_data 
    cipher = Cipher(algorithms.Camellia(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    #Step 2 : Unpad the padded part
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext)
    plaintext = message +unpadder.finalize()
    return plaintext
message = b'Hello world bhow are you foinklfdgd' #message not of 16, concretely 48 Bytes long(minimum here)
print("Message -> ", message , "\nlength : ",len(message))
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
ct = Camellia_cbc_encrypt(message,key,iv)
print("Ciphertext after using Camellia encryption : ", ct)
print("Ciphertext decrypted using Camellia decryption : ",  Camellia_cbc_decrypt(ct,key,iv))

Message ->  b'Hello world bhow are you foinklfdgd' 

length :  35

Ciphertext after using Camellia encryption :  b"8\xa3C'\xa4&[JTQ\xe5Lg\xbc\xf90f+\x19l\x14cmz\xf3`A\xd1V\x9b\x0f\xc3\x93\x91\xbd5>^<\xdbQ0\x04\xfe\xb0\xaa\x85}"

Ciphertext decrypted using Camellia decryption :  b'Hello world bhow are you foinklfdgddgd'


## 1.7 EXCHANGING SECRET MESSAGES WITH A PARTNER

### Task 1.7.1 Exchanging secret messages

#### TASK

<mark>Choose a partner among your classmates and exchange with him/her secret messages. You’ll need to send the key and other needed parameters along with the ciphertext. The receiver will also need to know the used algorithm, operation mode and key/block size if there are available more than one.</mark>

**Notice that in real life keys should NEVER be sent UNPROTECTED.**

# 2. Symmetric Stream Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric stream ciphers. First, you’ll use the CHACHA20 cipher and then you’ll use the pylfsr library to generate pseudorandom key streams using LFSRs.

## 2.1 CHACHA20 STREAM CIPHER

### Task 2.1.1 CHACHA20 encryption/decryption functions

#### TASK

Now read the information and take the code provided as example in the following web page section: 
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.ChaCha20

Take the code in the example and use it as base to create the functions whose headers are defined next. Test your functions with the data and results of Code Snippet 29 and Code Snippet 30. 

In [140]:
def chacha20_encrypt(message, key, nonce):
    from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
    algorithm = algorithms.ChaCha20(key, nonce)
    cipher = Cipher(algorithm, mode=None)
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message)
    return ciphertext

def chacha20_encrypt_rnd(message, key=None, nonce=None):
    if key is None:
        key = os.urandom(32)    # key must have 256 bits
        from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
        algorithm = algorithms.ChaCha20(key, nonce)
        cipher = Cipher(algorithm, mode=None)
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(message)
    return ciphertext
    if nonce is None:
        nonce = os.urandom(16)   # nonce must have 128 bits
        from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
        algorithm = algorithms.ChaCha20(key, nonce)
        cipher = Cipher(algorithm, mode=None)
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(message)
    return ciphertext, key, nonce;

def chacha20_decrypt(ciphertext, key, nonce):
    decryptor = cipher.decryptor()
    decryptor.update(ct)    
    return plaintext

#### CHECK

In [145]:
# Check your code
message = b'a secret message'


print("message :", message)
print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

print("ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce))
print("hex_ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce).hex())

print("plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce))
print("hex_plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce).hex())

print("ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce))
print("hex_ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce).hex())

print("plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce))
print("hex_plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce).hex())


message : b'a secret message'

other_message : b'This message is important'

key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a

nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

hex_nonce = f42489ab7ba42d603ade71b56d1bd398

ciphertext chacha20 message= b'Q\x02\x96\x92\xed\xc1\x90-\x97r\xaa/.\xb3`\xfd'

hex_ciphertext chacha20 message= 51029692edc1902d9772aa2f2eb360fd

plaintext chacha20 message= b'Hello world bhow are you foinklfdgddgd'

hex_plaintext chacha20 message= 48656c6c6f20776f726c642062686f772061726520796f7520666f696e6b6c66646764646764

ciphertext chacha20 other_message= b'dJ\x8c\x84\xae\xde\x90*\xc4~\xa89}\xbbt\xb8\x19G\xa7!\xac7\xed8z'

hex_ciphertext chacha20 other_message= 644a8c84aede902ac47ea8397dbb74b81947a721ac37ed387a

plaintext chacha20 other_message= b'Hello world bhow are you foinklfdgddgd'

hex_plaintext chacha20 other_message= 48656

You should obtain the following result:
<code>
message : b'a secret message'
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'</code>

<code>hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext chacha20 message= b'Q\x02\x96\x92\xed\xc1\x90-\x97r\xaa/.\xb3`\xfd'
hex_ciphertext chacha20 message= 51029692edc1902d9772aa2f2eb360fd
plaintext chacha20 message= b'a secret message'
hex_plaintext chacha20 message= 6120736563726574206d657373616765
ciphertext chacha20 other_message= b'dJ\x8c\x84\xae\xde\x90*\xc4~\xa89}\xbbt\xb8\x19G\xa7!\xac7\xed8z'
hex_ciphertext chacha20 other_message= 644a8c84aede902ac47ea8397dbb74b81947a721ac37ed387a
plaintext chacha20 other_message= b'This message is important'
hex_plaintext chacha20 other_message= 54686973206d65737361676520697320696d706f7274616e74  
</code>

## 2.2 GENERATING KEYSTREAMS WITH LFSRS

Now you’ll make use of the pylfsr library (https://lfsr.readthedocs.io/en/latest/index.html and https://pypi.org/project/pylfsr/). This library generates pseudorandom bit streams using, among other generators, LFSRs. It requires to have installed the numpy library, and in the case that you are using pylfsr’s graphical functions, the matplotlib library. 

### Task 2.2.1 Generating a key stream

Have a look at the information presented in web page https://lfsr.readthedocs.io/en/latest/index.html. 
Consider the example “Example 2: 5-bit LFSR with custum state and feedback polynomial”: https://lfsr.readthedocs.io/en/latest/LFSR_Examples.html#example-2-5-bit-lfsr-with-custum-state-and-feedback-polynomial 

#### TASK

<mark>Using the code in the example, generate 10 bits using primitive polynomial $x^5 + x^4 + x^3+ x^2 + 1$</mark>

In [148]:
# from pylfsr import LFSR

state = [0, 0, 0, 1, 0]   # seed = S1 S2 S3 S4 S5 
fpoly = [5,4,3,2]         # polynomial C5=1 C4=1 C3=1 C2=1 C1=0
L = LFSR(fpoly=fpoly, initstate=state, verbose=True)  # LFSR initialization
L.info()                  # prints information of the configured LFSR
tempseq = L.runKCycle(10) # generation of 10 bits 

5 bit LFSR with feedback polynomial  x^5 + x^4 + x^3 + x^2 + 1

Expected Period (if polynomial is primitive) =  31

Current :

 State        :  [0 0 0 1 0]

 Count        :  0

 Output bit   :  -1

 feedback bit :  -1

S:  [0 0 0 1 0]

S:  [1 0 0 0 1]

S:  [1 1 0 0 0]

S:  [1 1 1 0 0]

S:  [0 1 1 1 0]

S:  [1 0 1 1 1]

S:  [1 1 0 1 1]

S:  [1 1 1 0 1]

S:  [1 1 1 1 0]

S:  [1 1 1 1 1]


#### CHECK

The generated sequence is returned to the variable <code>tempseq</code>. Print its value and check that you obtain the same result as shown next: <code> [0 1 0 0 0 1 1 1 0 1] </code>

In [149]:
# YOUR CODE HERE
print(tempseq)

[0 1 0 0 0 1 1 1 0 1]


### Task 2.2.2 Checking that you generate the same keystream 

#### TASK

<mark>Make sure you understand how the class works by checking by hand that the code obtains the same 10-bit sequence that you compute. Use the same parameters:</mark>
- <mark>poly(x) = $x^5 + x^4 + x^3+ x^2 + 1 $</mark>
- <mark>seed = [S1 S2 S3 S4 S5] = [0, 0, 0, 1, 0]</mark>